In [27]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [28]:
more_features = False
use_staking = False
use_kernal_params = True
do_augment = True
fast = False

train_df = pd.read_csv('input/train' + ('_more_features' if more_features else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_more_features' if more_features else '') + '.csv')

do_lda = False

fix_data_skew = False

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [29]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [30]:
_, X_bottomhalf, _, y_bottomhalf = train_test_split(X, y, test_size=0.8, random_state=10)

In [31]:
if use_staking:
    X = X_bottomhalf
    y = y_bottomhalf

In [32]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [33]:
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.01,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8990901412442585,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample': 0.8054415526396443,
    'min_child_weight': 38.138072621096654,
    'num_threads': 4,
    'max_bin': 483
}

if use_kernal_params:
    # https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment (lb: 0.901)
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 13,
        "learning_rate" : 0.01,
        "bagging_freq": 5,
        "bagging_fraction" : 0.4,
        "feature_fraction" : 0.05,
        "min_data_in_leaf": 80,
        "min_sum_heassian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

In [34]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [35]:
oof = train_df[['ID_code', 'target']]
oof['predict'] = 0

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
predictions = test_df[['ID_code']]
val_aucs = []

In [37]:
score = 0.0
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    N = 20
    
    if not do_augment:
        N = 1
    
    p_valid,yp = 0,0
    for i in range(N):
        if do_augment:
            X_t, y_t = augment(X_train.values, y_train.values)
            X_t = pd.DataFrame(X_t)
            X_t = X_t.add_prefix('var_')

            trn_data = lgb.Dataset(X_t, label=y_t)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        else:
            trn_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        
        evals_result = {}
        
        early_stopping_rounds = 800 if fast else 3000
        
        lgb_clf = lgb.train(params,
                        trn_data,
                        num_boost_round=2000000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=early_stopping_rounds,
                        verbose_eval = 300,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    
    oof['predict'][valid_index] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold_n+1)] = yp/N

Fold 0 started at Thu Mar 28 08:28:46 2019
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.876491	valid_1's auc: 0.867014
[600]	training's auc: 0.887824	valid_1's auc: 0.876898
[900]	training's auc: 0.892475	valid_1's auc: 0.881427
[1200]	training's auc: 0.895384	valid_1's auc: 0.883875
[1500]	training's auc: 0.898106	valid_1's auc: 0.88604
[1800]	training's auc: 0.900636	valid_1's auc: 0.887993
[2100]	training's auc: 0.902817	valid_1's auc: 0.889695
[2400]	training's auc: 0.904681	valid_1's auc: 0.891035
[2700]	training's auc: 0.906681	valid_1's auc: 0.892392
[3000]	training's auc: 0.908279	valid_1's auc: 0.893523
[3300]	training's auc: 0.909817	valid_1's auc: 0.894422
[3600]	training's auc: 0.911261	valid_1's auc: 0.895127
[3900]	training's auc: 0.912682	valid_1's auc: 0.895872
[4200]	training's auc: 0.913905	valid_1's auc: 0.896614
[4500]	training's auc: 0.915016	valid_1's auc: 0.897198
[4800]	training's auc: 0.916091	valid_1's auc: 0.897736
[

[13200]	training's auc: 0.935984	valid_1's auc: 0.900291
[13500]	training's auc: 0.936591	valid_1's auc: 0.900262
Early stopping, best iteration is:
[10798]	training's auc: 0.931134	valid_1's auc: 0.90046
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.877368	valid_1's auc: 0.865982
[600]	training's auc: 0.888029	valid_1's auc: 0.876055
[900]	training's auc: 0.892852	valid_1's auc: 0.880972
[1200]	training's auc: 0.895615	valid_1's auc: 0.883319
[1500]	training's auc: 0.898261	valid_1's auc: 0.885579
[1800]	training's auc: 0.900765	valid_1's auc: 0.887659
[2100]	training's auc: 0.903028	valid_1's auc: 0.889306
[2400]	training's auc: 0.904923	valid_1's auc: 0.890621
[2700]	training's auc: 0.906949	valid_1's auc: 0.891988
[3000]	training's auc: 0.908528	valid_1's auc: 0.893082
[3300]	training's auc: 0.9101	valid_1's auc: 0.894075
[3600]	training's auc: 0.911499	valid_1's auc: 0.894866
[3900]	training's auc: 0.912902	valid_1's auc: 0.895655
[4200]	t

[14100]	training's auc: 0.937596	valid_1's auc: 0.900231
[14400]	training's auc: 0.938176	valid_1's auc: 0.900183
[14700]	training's auc: 0.938745	valid_1's auc: 0.90022
Early stopping, best iteration is:
[11850]	training's auc: 0.933114	valid_1's auc: 0.900487
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.876435	valid_1's auc: 0.866182
[600]	training's auc: 0.887371	valid_1's auc: 0.876295
[900]	training's auc: 0.89204	valid_1's auc: 0.880684
[1200]	training's auc: 0.894967	valid_1's auc: 0.883131
[1500]	training's auc: 0.897678	valid_1's auc: 0.885297
[1800]	training's auc: 0.900221	valid_1's auc: 0.887273
[2100]	training's auc: 0.902496	valid_1's auc: 0.889129
[2400]	training's auc: 0.904458	valid_1's auc: 0.890619
[2700]	training's auc: 0.906403	valid_1's auc: 0.891948
[3000]	training's auc: 0.908013	valid_1's auc: 0.893088
[3300]	training's auc: 0.909605	valid_1's auc: 0.894111
[3600]	training's auc: 0.911009	valid_1's auc: 0.894951
[3900]

[13800]	training's auc: 0.936915	valid_1's auc: 0.900091
[14100]	training's auc: 0.937489	valid_1's auc: 0.900095
Early stopping, best iteration is:
[11274]	training's auc: 0.931826	valid_1's auc: 0.900319
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.87679	valid_1's auc: 0.865871
[600]	training's auc: 0.88757	valid_1's auc: 0.87605
[900]	training's auc: 0.892527	valid_1's auc: 0.880908
[1200]	training's auc: 0.895429	valid_1's auc: 0.883537
[1500]	training's auc: 0.898232	valid_1's auc: 0.885725
[1800]	training's auc: 0.900744	valid_1's auc: 0.887868
[2100]	training's auc: 0.902946	valid_1's auc: 0.889698
[2400]	training's auc: 0.904914	valid_1's auc: 0.891001
[2700]	training's auc: 0.906866	valid_1's auc: 0.892336
[3000]	training's auc: 0.908485	valid_1's auc: 0.893436
[3300]	training's auc: 0.910068	valid_1's auc: 0.894393
[3600]	training's auc: 0.911486	valid_1's auc: 0.895221
[3900]	training's auc: 0.912827	valid_1's auc: 0.896041
[4200]	t

[15600]	training's auc: 0.940372	valid_1's auc: 0.900126
[15900]	training's auc: 0.940932	valid_1's auc: 0.900167
Early stopping, best iteration is:
[13035]	training's auc: 0.935416	valid_1's auc: 0.900429
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.876346	valid_1's auc: 0.866562
[600]	training's auc: 0.88747	valid_1's auc: 0.876072
[900]	training's auc: 0.892261	valid_1's auc: 0.881021
[1200]	training's auc: 0.895263	valid_1's auc: 0.883547
[1500]	training's auc: 0.897997	valid_1's auc: 0.885889
[1800]	training's auc: 0.900487	valid_1's auc: 0.887845
[2100]	training's auc: 0.902701	valid_1's auc: 0.889632
[2400]	training's auc: 0.904619	valid_1's auc: 0.890988
[2700]	training's auc: 0.906625	valid_1's auc: 0.892376
[3000]	training's auc: 0.908244	valid_1's auc: 0.893478
[3300]	training's auc: 0.909796	valid_1's auc: 0.894391
[3600]	training's auc: 0.911207	valid_1's auc: 0.895183
[3900]	training's auc: 0.912613	valid_1's auc: 0.896027
[4200]

...

[3900]	training's auc: 0.912446	valid_1's auc: 0.892174
[4200]	training's auc: 0.913692	valid_1's auc: 0.893042
[4500]	training's auc: 0.914814	valid_1's auc: 0.893675
[4800]	training's auc: 0.915899	valid_1's auc: 0.894274
[5100]	training's auc: 0.916913	valid_1's auc: 0.89476
[5400]	training's auc: 0.917839	valid_1's auc: 0.895208
[5700]	training's auc: 0.918731	valid_1's auc: 0.895513
[6000]	training's auc: 0.919586	valid_1's auc: 0.895849
[6300]	training's auc: 0.920413	valid_1's auc: 0.896081
[6600]	training's auc: 0.921223	valid_1's auc: 0.89634
[6900]	training's auc: 0.921989	valid_1's auc: 0.896559
[7200]	training's auc: 0.922705	valid_1's auc: 0.896756
[7500]	training's auc: 0.923436	valid_1's auc: 0.896916
[7800]	training's auc: 0.924145	valid_1's auc: 0.897054
[8100]	training's auc: 0.924839	valid_1's auc: 0.897101
[8400]	training's auc: 0.92553	valid_1's auc: 0.897126
[8700]	training's auc: 0.926196	valid_1's auc: 0.897223
[9000]	training's auc: 0.92687	valid_1's auc: 0.897

[3000]	training's auc: 0.908499	valid_1's auc: 0.889676
[3300]	training's auc: 0.910021	valid_1's auc: 0.890856
[3600]	training's auc: 0.911391	valid_1's auc: 0.891819
[3900]	training's auc: 0.91275	valid_1's auc: 0.892704
[4200]	training's auc: 0.914007	valid_1's auc: 0.8935
[4500]	training's auc: 0.915143	valid_1's auc: 0.894165
[4800]	training's auc: 0.916248	valid_1's auc: 0.89479
[5100]	training's auc: 0.917276	valid_1's auc: 0.895308
[5400]	training's auc: 0.9182	valid_1's auc: 0.89567
[5700]	training's auc: 0.919075	valid_1's auc: 0.896036
[6000]	training's auc: 0.919922	valid_1's auc: 0.896279
[6300]	training's auc: 0.920735	valid_1's auc: 0.896493
[6600]	training's auc: 0.921541	valid_1's auc: 0.89675
[6900]	training's auc: 0.922306	valid_1's auc: 0.896883
[7200]	training's auc: 0.923053	valid_1's auc: 0.897077
[7500]	training's auc: 0.923768	valid_1's auc: 0.897275
[7800]	training's auc: 0.924468	valid_1's auc: 0.897395
[8100]	training's auc: 0.925183	valid_1's auc: 0.897448


[3900]	training's auc: 0.912475	valid_1's auc: 0.892431
[4200]	training's auc: 0.913721	valid_1's auc: 0.893262
[4500]	training's auc: 0.914877	valid_1's auc: 0.893942
[4800]	training's auc: 0.915958	valid_1's auc: 0.894554
[5100]	training's auc: 0.916962	valid_1's auc: 0.895049
[5400]	training's auc: 0.917867	valid_1's auc: 0.895443
[5700]	training's auc: 0.918763	valid_1's auc: 0.895745
[6000]	training's auc: 0.919626	valid_1's auc: 0.895973
[6300]	training's auc: 0.920441	valid_1's auc: 0.896173
[6600]	training's auc: 0.9212	valid_1's auc: 0.896344
[6900]	training's auc: 0.921975	valid_1's auc: 0.896485
[7200]	training's auc: 0.922743	valid_1's auc: 0.896665
[7500]	training's auc: 0.923485	valid_1's auc: 0.896785
[7800]	training's auc: 0.924202	valid_1's auc: 0.896943
[8100]	training's auc: 0.924907	valid_1's auc: 0.896992
[8400]	training's auc: 0.925581	valid_1's auc: 0.897055
[8700]	training's auc: 0.926256	valid_1's auc: 0.897107
[9000]	training's auc: 0.926928	valid_1's auc: 0.8

[4800]	training's auc: 0.916127	valid_1's auc: 0.894334
[5100]	training's auc: 0.917144	valid_1's auc: 0.894819
[5400]	training's auc: 0.918081	valid_1's auc: 0.895237
[5700]	training's auc: 0.918927	valid_1's auc: 0.895548
[6000]	training's auc: 0.91976	valid_1's auc: 0.895829
[6300]	training's auc: 0.920569	valid_1's auc: 0.896074
[6600]	training's auc: 0.921363	valid_1's auc: 0.896319
[6900]	training's auc: 0.922136	valid_1's auc: 0.89648
[7200]	training's auc: 0.922878	valid_1's auc: 0.896631
[7500]	training's auc: 0.923607	valid_1's auc: 0.896749
[7800]	training's auc: 0.924309	valid_1's auc: 0.896942
[8100]	training's auc: 0.925022	valid_1's auc: 0.897049
[8400]	training's auc: 0.925698	valid_1's auc: 0.897096
[8700]	training's auc: 0.926378	valid_1's auc: 0.897127
[9000]	training's auc: 0.927041	valid_1's auc: 0.897142
[9300]	training's auc: 0.927684	valid_1's auc: 0.897111
[9600]	training's auc: 0.928342	valid_1's auc: 0.897149
[9900]	training's auc: 0.928986	valid_1's auc: 0.8

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

In [38]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.901776835, std: 0.003131583. All auc: 0.901767779.


In [39]:
# submission
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('lgb_all_predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions['target']
sub_df.to_csv("lgb_submission.csv", index=False)
oof.to_csv('lgb_oof.csv', index=False)

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
